In [ ]:
import os
import openai
import time
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, classification_report

openai.api_key = os.getenv("OPENAI_API_KEY")

import json
import numpy as np

np.random.seed(0)
features_valid = json.load(open("../data/valid_subset_text.json"))
features_rest = json.load(open("../dataset_bias/valid_subset_rest_narrative.json"))

gen_examplars_res = np.load("../data/examplars_gda_rest.npy", allow_pickle=True)

def compare(d1, d2):
    return all(d1[key] == d2[key] for key in d1)

features_ori = [item for item in features_valid if all(not compare(d, item) for d in features_rest)]

In [2]:
all_examplars_conflict = np.load("../data/examplars_gda_conflict.npy", allow_pickle=True)
all_examplars_rest = np.load("../data/examplars_gda_rest.npy", allow_pickle=True)

all_examplars_new = [[] for i in range(len(features_valid))]

rest_2_all_idx = []

for i in range(len(features_rest)):
    rest_2_all_idx.append([])
    for j in range(len(features_valid)):
        if compare(features_valid[j], features_rest[i]):
            rest_2_all_idx[-1].append(j)
for i in range(len(features_rest)):
    for idx in rest_2_all_idx[i]:
        all_examplars_new[idx] = all_examplars_rest[i]
        
conflict_2_all_idx = []

for i in range(len(features_ori)):
    conflict_2_all_idx.append([])
    for j in range(len(features_valid)):
        if compare(features_valid[j], features_ori[i]):
            conflict_2_all_idx[-1].append(j)
for i in range(len(features_ori)):
    for idx in conflict_2_all_idx[i]:
        all_examplars_new[idx] = all_examplars_conflict[i]

In [5]:
np.save("../data/valid_gda_examplars", all_examplars_new)

In [6]:
selected_subsubset = np.load("subsubset_idx_200.npy", allow_pickle=True)

In [12]:
three_shot_examplars = open("prompts/prompt_1_icl_examplar_3_shot_1.txt").readlines()

In [28]:


sampled_two_shot_exampalrs = ['Given the context:\n' + item for i, item in enumerate("".join(three_shot_examplars).split('Given the context:\n')[1:]) if (i + 1) % 3 !=0 ]



In [39]:
def merge_gda_examplars(gda_list):
    assert len(gda_list) == 4
    return sampled_two_shot_exampalrs[:2] + [gda_list[0]+"\n\n"] + sampled_two_shot_exampalrs[2:4] + [gda_list[1]+"\n\n"] \
    + sampled_two_shot_exampalrs[4:6] + [gda_list[2]+"\n\n"] + sampled_two_shot_exampalrs[6:8] + [gda_list[3]+"\n\n"]

In [40]:
merge_gda_examplars(all_examplars_new[0])

['Given the context:\nThe thrift announced the plan Aug. 21. Among other reasons, high fees regulators imposed on certain transfers of thrift deposits to commercial banks "substantially altered the economics of the transaction for both parties," Poughkeepsie said. Additionally, the bank is increasing its loan-loss reserves for the third quarter by $8.5 million before taxes.\n\nQ: What\'s the temporal relation between the event "altered" and "said"? \nChoice A: altered happens before said. \nChoice B: altered happens after said. \nChoice C: altered happens during said. \nChoice D: unknown. \nAnswer only with A, B, C, or D. \n\nA: Choice A\n\n',
 'Given the context:\nThe U.N. Security Council on Aug. 6 ordered a global embargo on trade with Iraq as punishment for seizing Kuwait. Bush then sent U.S. troops and planes to Saudi Arabia, saying Iraqi troops massed nearby in Kuwait threatened Saudi Arabia and its vast oil reserves. Twelve Arab League nations voted for an Arab force to protect 

In [42]:
threeshot_cda_results_1 = []

# selected_subsubset = np.random.permutation(len(features_valid))[:200]
# np.save("subsubset_idx_200", selected_subsubset)
selected_subsubset = np.load("subsubset_idx_200.npy", allow_pickle=True)

for i in tqdm(selected_subsubset):

    item = features_valid[i]
    context = item['text'].replace("[CLS]", "").replace("[SEP]", "").strip()
    e1 = item['e1']
    e2 = item['e2']
    
    examplar = ''.join(merge_gda_examplars(all_examplars_new[i]))
    
    prompt = "Given the context:\n" + context + f"\n\nQ: What's the temporal relation between the event \"{e1}\" and \"{e2}\"? \nChoice A: {e1} happens before {e2}. \nChoice B: {e1} happens after {e2}. \nChoice C: {e1} happens during {e2}. \nChoice D: unknown. \nAnswer only with A, B, C, or D. \n\nA: Choice "
#     print(prompt)
    while True:
        try:
            threeshot_cda_results_1.append(openai.Completion.create(
                        model="text-davinci-003",
                        prompt=examplar + "\n\n" + prompt,
                        max_tokens=20,
                        temperature=0
            ))
            break
        except:
            time.sleep(10)
    time.sleep(2)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [11:32<00:00,  3.46s/it]


In [43]:
convert_dict = {'A':0, 'B':1, 'C':2, 'D':3}
convert_dict_rev = {0:'A', 1:'B', 2:'C', 3:'D'}
def parse_result(ans):
    return convert_dict.get(ans.upper(), 3)


labels_selected = [features_valid[i]['labels'][0] for i in selected_subsubset]
threeshot_preds_1 =  [parse_result(item['choices'][0]['text'].strip()) for item in threeshot_cda_results_1]

print(f1_score(labels_selected, threeshot_preds_1, average='macro'), f1_score(labels_selected, threeshot_preds_1, average="micro"))


0.26348359035191776 0.515


In [49]:
threeshot_cda_results_2 = []

# selected_subsubset = np.random.permutation(len(features_valid))[:200]
# np.save("subsubset_idx_200", selected_subsubset)
selected_subsubset = np.load("subsubset_idx_200.npy", allow_pickle=True)

for i in tqdm(selected_subsubset):

    item = features_valid[i]
    context = item['text'].replace("[CLS]", "").replace("[SEP]", "").strip()
    e1 = item['e1']
    e2 = item['e2']
    
    merged_results = merge_gda_examplars(all_examplars_new[i])
    examplar = ''.join(merged_results[:2] + merged_results[3:])
    
    prompt = "Given the context:\n" + context + f"\n\nQ: What's the temporal relation between the event \"{e1}\" and \"{e2}\"? \nChoice A: {e1} happens before {e2}. \nChoice B: {e1} happens after {e2}. \nChoice C: {e1} happens during {e2}. \nChoice D: unknown. \nAnswer only with A, B, C, or D. \n\nA: Choice "
#     print(prompt)
    while True:
        try:
            threeshot_cda_results_2.append(openai.Completion.create(
                        model="text-davinci-003",
                        prompt=examplar + "\n\n" + prompt,
                        max_tokens=20,
                        temperature=0
            ))
            break
        except:
            time.sleep(10)
    time.sleep(2)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [10:56<00:00,  3.28s/it]


In [50]:
convert_dict = {'A':0, 'B':1, 'C':2, 'D':3}
convert_dict_rev = {0:'A', 1:'B', 2:'C', 3:'D'}
def parse_result(ans):
    return convert_dict.get(ans.upper(), 3)


labels_selected = [features_valid[i]['labels'][0] for i in selected_subsubset]
threeshot_preds_2 =  [parse_result(item['choices'][0]['text'].strip()) for item in threeshot_cda_results_2]

print(f1_score(labels_selected, threeshot_preds_2, average='macro'), f1_score(labels_selected, threeshot_preds_2, average="micro"))


0.2195288494366678 0.47
